# 유튜브 api 크롤링 결과 분석기

In [ ]:
# 크롤링 결과 가져오기

f = open('./crawling_result_text/gamst.txt', mode='rt', encoding='utf-8')

strings = f.read()

f.close()

print(strings)

In [ ]:
# 정규표현식 활용 말뭉치 정리

import re


stop_special_char = re.sub(pattern='[가-힣 a-zA-Z\d\.\n]+', repl='', string=strings)
stop_special_char_pattern = '[' + '\\'.join(set(stop_special_char)) + ']'

print(stop_special_char_pattern)

new_strings = re.sub(pattern=stop_special_char_pattern, repl='',string=strings)
new_strings = re.sub(pattern='\s{2,}',repl=' ',string=new_strings)
new_strings = re.sub(pattern='[\n]+',repl=' ',string=new_strings)
new_strings = re.sub(pattern='[a-zA-Z\,]+',repl='',string=new_strings)
new_strings = re.sub(pattern='[0-9]+', repl='', string=new_strings)
new_strings = re.sub(pattern='\s{2,}',repl=' ',string=new_strings)
new_strings = re.sub(pattern='롤', repl='리그오브레전드', string=new_strings)
new_strings = re.sub(pattern='아프리카', repl='아프리카티비', string=new_strings)
new_strings = re.sub(pattern='아프리카티비티비', repl='아프리카티비', string=new_strings)
# new_strings = [ line.strip() for line in new_strings.split('.') if len(line) > 4 ]
print(new_strings)

In [ ]:
#품사 태깅

from konlpy.tag import Hannanum

hannanum = Hannanum()

print(hannanum.nouns(new_strings))

In [ ]:
#불용어사전(자체 제작)
stopwords = ['라', '중', '이', '때문', '지', '이상', '등', '수', '것', '시작', '부분', '당시', '경우', '이후', '오브', '리그', '편', '위', '정도', '활동', '전', 
    '한', '도중', '자체', '경기', '방송이','때','녹두','이유','문단을','대부분','번','일','리','두','초','나','문단','문','방송','적','후','단어','게임','유튜브','영상','말','역사','시청자들','본인','이전','시청자','2020년','2018년','2019년','2017년','2016년','업로드','콘텐츠','듯','현재','명','들','사람','1','개','관련','채널','모습','주','거','내','사이','자신','유튜버','컨텐츠','생각'] +['1월','2월','3월','4월','5월','6월','7월','8월','9월','10월','11월','12월'] + ['감스트', '비감', '인직'] + ['진짜'] + ['아', '휴', '아이구', '아이쿠', '아이고','어','나','우리','저희','따라','의해','을','를','에','의','가','으로','로','에게','뿐이다','저','지말고','다','뿐','허','헉','곧','이다','도','부터','모','등','조금','좀','대','종','망','그','데','에','애','제','장']

In [ ]:
#워드 클라우드 생성

from collections import Counter
import random
import math

from konlpy.tag import Hannanum
from konlpy.tag import Twitter
import pytagcloud
import webbrowser

r = lambda: random.randint(0,255)
color = lambda: (r(),r(),r())

def get_tags(text, ntags=100, multiplier=10):
    h = Hannanum()
    nouns = h.nouns(text)
    unique_nouns = set(nouns)
    for word in unique_nouns:
        if word in stopwords:
            while word in nouns:
                nouns.remove(word)

    count = Counter(nouns)
    print(count)
    return [{ 'color': color(), 'tag': n, 'size': int(math.sqrt(c)*multiplier) }\
                for n, c in count.most_common(ntags)]



def get_tags_twitter(text, ntags=100, multiplier=5):
    t =Twitter()
    nouns = t.nouns(text)
    unique_nouns = set(nouns)
    for word in unique_nouns:
        if word in stopwords:
            while word in nouns:
                nouns.remove(word)

    count = Counter(nouns)
    return [{ 'color': color(), 'tag': n, 'size': int(c*multiplier) }\
                for n, c in count.most_common(ntags)]


def draw_cloud(tags, filename, fontname='Korean', size=(1600, 1200)):
    pytagcloud.create_tag_image(tags, filename, fontname=fontname, size=size)
    webbrowser.open( os.path.relpath(filename) )


tags = get_tags(new_strings)
# print(tags)
draw_cloud(tags,'./crawling_result_wordcloud/WC_You_gamst.png')


In [ ]:
#파일 하나 열어서 전처리 후 단어 추출 후 다시 문장으로 만드는 함수

import re
from konlpy.tag import Hannanum

def make_clean_sting(filename):
    # 파일 읽기
    with open(filename, mode='rt', encoding='utf-8') as f:
        strings = f.read()
    
    #전처리
    stop_special_char = re.sub(pattern='[가-힣 a-zA-Z\d\.\n]+', repl='', string=strings)
    stop_special_char_pattern = '[' + '\\'.join(set(stop_special_char)) + ']'

    new_strings = re.sub(pattern=stop_special_char_pattern, repl='',string=strings)
    new_strings = re.sub(pattern='\s{2,}',repl=' ',string=new_strings)
    new_strings = re.sub(pattern='[\n]+',repl=' ',string=new_strings)
    new_strings = re.sub(pattern='[a-zA-Z\,]+',repl='',string=new_strings)
    new_strings = re.sub(pattern='[0-9]+', repl='', string=new_strings)
    new_strings = re.sub(pattern='\s{2,}',repl=' ',string=new_strings)
    new_strings = re.sub(pattern='롤', repl='리그오브레전드', string=new_strings)
    new_strings = re.sub(pattern='아프리카', repl='아프리카티비', string=new_strings)
    new_strings = re.sub(pattern='아프리카티비티비', repl='아프리카티비', string=new_strings)

    # 명사 추출
    hannanum = Hannanum()
    string2noun = hannanum.nouns(new_strings)
    
    #다시 합치기
    noun2string = " ".join(string2noun)


    print('task [%s] clear' % filename)
    return noun2string







In [ ]:
#doc set 만들기

gamst = make_clean_sting('./crawling_result_text/gamst.txt')
handongsuk = make_clean_sting('./crawling_result_text/handongsuk.txt')
testerhoon = make_clean_sting('./crawling_result_text/testerhoon.txt')

doc_list = []
doc_list.append(gamst)
doc_list.append(handongsuk)
doc_list.append(testerhoon)


In [93]:
# tf-idf 

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

x = tfidf.fit_transform(doc_list)
word_list = tfidf.get_feature_names()
# print(x)
# print(x.data)
# print(x.indices)
# print(x.indptr)


new_data_list = []

for i in range(0,x.indptr[3]):
    new_data = [0,0]
    new_data[0] = x.data[i]
    new_data[1] = x.indices[i]
    new_data_list.append(new_data)

top10_list = [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]
top10_num_list = [0,0,0,0,0,0,0,0,0,0]


#첫번째 문서 탑10
for i in range(0,x.indptr[1]):
    current_value = new_data_list[i][0]
    if top10_list[9][0] < current_value:
        top10_list[9] = new_data_list[i]
        top10_list.sort(key=lambda x:x[0],reverse=True)
        

for i in top10_list:
    print(word_list[i[1]])

top10_list = [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]

print('\n')
#두번째 문서 탑10
for i in range(x.indptr[1],x.indptr[2]):
    current_value = new_data_list[i][0]
    if top10_list[9][0] < current_value:
        top10_list[9] = new_data_list[i]
        top10_list.sort(key=lambda x:x[0],reverse=True)
        

for i in top10_list:
    print(word_list[i[1]])

top10_list = [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]

print('\n')
#세번째 문서 탑10
for i in range(x.indptr[2],x.indptr[3]):
    current_value = new_data_list[i][0]
    if top10_list[9][0] < current_value:
        top10_list[9] = new_data_list[i]
        top10_list.sort(key=lambda x:x[0],reverse=True)
        

for i in top10_list:
    print(word_list[i[1]])





        
     






감스트
아프리카티비
진짜
오세블리
인직
김인직
감튜브
비감
방송영상
방송


동수칸
진짜
버튼
트위치
한동숙
구독
펨플릭스
댓글
카페
방송국


테스터훈
장인초대석
바로가
진짜
장인
페이스북
게임
코리안메타
뉴메타
비즈니스
